In [1]:
import time
import plotly.express as px
import pandas as pd
import datetime as dt
import plotly.graph_objects as go


df_solution = pd.read_excel("data/TUEdatav1.xlsx", sheet_name="InitialSolution")
df_steps = pd.read_excel("data/TUEdatav1.xlsx", sheet_name="Steps")
df_runs = pd.read_excel("data/TUEdatav1.xlsx", sheet_name="Runs")
df_runs = pd.melt(df_runs, id_vars=['RunId'], value_vars=['Step1', 'Step2', 'Step3'], var_name="StepNr", value_name="StepId")

df_solution = df_solution.join(df_steps[["StepId", "MetalType"]].set_index("StepId"), on="StepId")
df_solution["MetalType"] = df_solution["MetalType"].astype(str)

df_solution = df_solution.join(df_runs[["RunId", "StepId"]].set_index("StepId"), on="StepId")

# min_time = df_solution["StartDate_Seconds"].min()
df_solution["StartDate_Seconds_norm"] = (df_solution["StartDate_Seconds"])
df_solution["EndDate_Seconds_norm"] = (df_solution["EndDate_Seconds"])

start_date = pd.to_datetime("2023-11-06")
df_solution["Start"] = start_date + pd.to_timedelta(df_solution['StartDate_Seconds'], unit='s')
df_solution["End"] = start_date + pd.to_timedelta(df_solution['EndDate_Seconds'], unit='s')
df_solution["Machine"] = df_solution["StepId"].str.get(0)

mask_setup_times = df_solution["SetupTime_Hours"] > 0

#DataFrame for just the setup times
df_setup_times = df_solution[mask_setup_times].copy()
df_setup_times["End"] = df_setup_times["Start"] + pd.to_timedelta(df_setup_times["SetupTime_Hours"], unit='h')

#Shorten the "Step" into its actual Step time (excluding setup time_
df_solution["Start"] = df_solution["Start"] + pd.to_timedelta(df_solution["SetupTime_Hours"], unit='h')

actual_start_time = df_solution["Start"].min()

fig = px.timeline(df_solution,
                  x_start="Start",
                  x_end="End",
                  y="Machine",
                  color="MetalType",
                  hover_data=["StepId", "TooLate_Weeks", "SetupTime_Hours", "StartDate_Seconds", "StartDate_Seconds"],
                  custom_data=["RunId"],
                  range_x=(actual_start_time, actual_start_time + pd.to_timedelta(3600*24, unit='s')))
fig.update_yaxes(autorange="reversed", fixedrange=True)

fig_setup = px.timeline(df_setup_times,
                        x_start="Start",
                        x_end="End",
                        y="Machine",
                        color="MetalType",
                        hover_data=["StepId", "TooLate_Weeks", "SetupTime_Hours"],
                        color_discrete_map={metal_type: "silver" for metal_type in df_setup_times["MetalType"].unique()})

df_solution_lines = df_solution.copy()
df_solution_lines["Duration"] = (df_solution_lines["End"] - df_solution_lines["Start"]).dt.seconds
df_solution_lines["X"] = df_solution_lines["Start"] + pd.to_timedelta(df_solution_lines["Duration"]/2.0, unit='s')

fig_lines = px.line(df_solution_lines,
                    x="X",
                    y="Machine",
                    color="RunId",
                    color_discrete_map={x: "black" for x in df_solution_lines["RunId"].unique()})
fig_lines.update_traces(visible=False)

new_fig = go.Figure(data=fig.data + fig_setup.data + fig_lines.data, layout=fig.layout)
# new_fig.show("browser")

w = go.FigureWidget(new_fig)

def update_line_visibility(trace, points, selector):
    if len(points.point_inds) > 0:
        # print(points)
        steps = df_solution[(df_solution["End"] == points.xs[0]) & (df_solution["Machine"] == points.ys[0])].reset_index()
        if len(steps) > 0:
            run_id = steps["RunId"][0]
            # print(run_id)
            for i in range(len(w.data)):
                    d = w.data[i]
                    if d.name.startswith("R"):
                        if d.visible or run_id == d.name:
                            d.visible = run_id == d.name

for i in range(len(w.data)):
    if not w.data[i].name.startswith("R"):
        w.data[i].on_hover(update_line_visibility)
w

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'base': array([datetime.datetime(2023, 11, 7, 23, 52, 15),
                             datetime.datetime(2023, 11, 8, 0, 38, 1),
                             datetime.datetime(2023, 11, 8, 1, 23, 47),
                             datetime.datetime(2023, 11, 8, 2, 9, 33),
                             datetime.datetime(2023, 11, 8, 2, 55, 19),
                             datetime.datetime(2023, 11, 8, 3, 41, 5),
                             datetime.datetime(2023, 11, 11, 14, 7, 51),
                             datetime.datetime(2023, 11, 11, 16, 38, 29),
                             datetime.datetime(2023, 11, 11, 17, 24, 15),
                             datetime.datetime(2023, 11, 11, 18, 10, 1),
                             datetime.datetime(2023, 11, 11, 18, 55, 47),
                             datetime.datetime(2023, 11, 11, 19, 41, 33),
                             datetime.datetime(2023, 11, 12, 1, 2, 51),
 